# Results p. 2
## Fixation & Saccade Temporal Alignment
### (Comparing RTO & RTD between Detectors)

In [1]:
import copy

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_time_diffs as ctd
import analysis.statistics.channel_sdt as ch_sdt

# pio.renderers.default = "browser"

MATCHING_THRESHOLD = 20     # samples; matches with time-diff greater than this value are not considered "hits"

## Load Data
The `time_diffs` table describes the difference in onset/offset timings between the ground truth and the predicted fixations.  
The `thresholded_time_diffs` table is a subset of `time_diffs` where only the time-diffs whose absolute value is less than or equal to the `MATCHING_THRESHOLD` are kept.
#### (1) Fixation Data

In [2]:
fix_time_diffs = ctd.load(
    DATASET_NAME,
    PROCESSED_DATA_DIR,
    label=1,
    stimulus_type=STIMULUS_TYPE,
)
fix_thresholded_time_diffs = fix_time_diffs.map(lambda vals: np.fromiter(filter(lambda v: abs(v) <= MATCHING_THRESHOLD, vals), dtype=int))

#### (2) Saccade Data

In [3]:
sac_time_diffs = ctd.load(
    DATASET_NAME,
    PROCESSED_DATA_DIR,
    label=2,
    stimulus_type=STIMULUS_TYPE,
)
sac_thresholded_time_diffs = sac_time_diffs.map(lambda vals: np.fromiter(filter(lambda v: abs(v) <= MATCHING_THRESHOLD, vals), dtype=int))

## Compute Metrics

_Comments on the calculations:_  
(1) `num_gt_events` is the number of onsets/offsets across all trials, for the human annotator of every _(human, algorithm)_ pair.  
(2) `num_pred_events` is the number of onsets/offsets across all trials, for the algorithm of every _(human, algorithm)_ pair (or the _other human_ annotator).  
  
(3) `num_matches` is the number of matching onsets/offsets between the ground truth and the predicted events.  
  
(4) `hit_rate` (_recall_) is the percentage of matched GT onsets/offsets out of all GTs.  
(5) `ppv` (_precision_) is the percentage of matched PRED onsets/offsets out of all PREDs.  
(6) `f1` is the harmonic mean of `hit_rate` and `ppv`.
  
(7) `RTO` is the mean difference in onset/offset timings (measured in samples).  
(8) `RTD` is the standard deviation of difference in onset/offset timings (measured in samples).  

In [4]:
def calculate_alignment_metrics(td: pd.DataFrame, label: int) -> pd.DataFrame:
    num_gt_events, num_pred_events = _calc_num_events(label)
    aggregate_td = td.unstack(0).unstack(0).apply(np.hstack, axis=1).apply(lambda vals: vals[~np.isnan(vals)]).rename("time_diffs")
    aggregate_td = aggregate_td.reindex(u.sort_labelers(aggregate_td.index.get_level_values(u.PRED_STR)), level=u.PRED_STR)
    num_matches = aggregate_td.apply(len).rename('num_matches')     # number of matching onsets/offsets between the ground truth and the predicted events.
    
    hit_rate = (num_matches / num_gt_events).rename('hit_rate')
    ppv = (num_matches / num_pred_events).rename('ppv')
    f1 = (2 * (hit_rate * ppv) / (hit_rate + ppv)).rename('f1')
    rto = aggregate_td.apply(np.mean).rename('RTO')
    rtd = aggregate_td.apply(np.std).rename('RTD')
    
    alignment = pd.concat([num_gt_events, num_pred_events, num_matches, hit_rate, ppv, f1, rto, rtd], axis=1).unstack(
        [u.GT_STR, u.PRED_STR]).stack(0, future_stack=True)
    alignment.reindex(columns=u.sort_labelers(alignment.columns.get_level_values(u.PRED_STR).unique()), level=1)
    alignment.index.names = [peyes.constants.CHANNEL_TYPE_STR, peyes.constants.METRIC_STR]
    alignment.columns.names = [u.GT_STR, u.PRED_STR]
    return alignment


def _calc_num_events(label: int) -> (pd.Series, pd.Series):
    # Loads the channel-SDT data and extracts the number of GT and PRED events (onsets/offsets) for each labeler-detector pair.
    num_events = ch_sdt.load(
        dataset_name=DATASET_NAME,
        output_dir=PROCESSED_DATA_DIR,
        label=label,
        stimulus_type=STIMULUS_TYPE,
        channel_type=None,
    )
    num_events = num_events.loc[(slice(None), ['P', 'PP'], 0)]      # keep only the positive counts
    num_events = num_events.droplevel(peyes.constants.THRESHOLD_STR)
    num_events = num_events.stack([u.GT_STR, u.PRED_STR], future_stack=True).sum(axis=1)
    num_events = num_events.reorder_levels([u.GT_STR, u.PRED_STR, peyes.constants.METRIC_STR, peyes.constants.CHANNEL_TYPE_STR]).sort_index()
    
    num_gt_events = num_events.xs('P', level=peyes.constants.METRIC_STR).rename('num_gt')   # number of onsets/offsets across all trials, for the human annotator of every _(human, algorithm)_ pair.
    num_pred_events = num_events.xs('PP', level=peyes.constants.METRIC_STR).rename('num_pred')  # number of onsets/offsets across all trials, for the algorithm of every _(human, algorithm)_ pair (or the _other human_ annotator).
    return num_gt_events, num_pred_events

In [5]:
fix_alignment_metrics = calculate_alignment_metrics(fix_thresholded_time_diffs, 1)
sac_alignment_metrics = calculate_alignment_metrics(sac_thresholded_time_diffs, 2)

alignment_metrics = pd.concat([fix_alignment_metrics, sac_alignment_metrics], axis=0, keys=["fixation", "saccade"])
alignment_metrics.index.names = [peyes.constants.EVENT_STR] + alignment_metrics.index.names[1:]

#### Annotator RA

In [6]:
ra_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_onset_alignments = ra_onset_alignments.reindex(
    columns=u.sort_labelers(ra_onset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_onset_alignments

pred                          MN         ivt         ivvt         idt  \
event    metric                                                         
fixation num_gt       391.000000  563.000000   563.000000  563.000000   
         num_pred     404.000000  696.000000  1042.000000  358.000000   
         num_matches  375.000000  519.000000   549.000000  221.000000   
         hit_rate       0.959079    0.921847     0.975133    0.392540   
         ppv            0.928218    0.745690     0.526871    0.617318   
         f1             0.943396    0.824464     0.684112    0.479913   
         RTO           -0.917333   -4.019268    -0.400729  -12.058824   
         RTD            4.023411    4.421398     4.190335    5.842672   
saccade  num_gt       374.000000  552.000000   552.000000  552.000000   
         num_pred     377.000000  711.000000   888.000000  342.000000   
         num_matches  369.000000  517.000000   530.000000  288.000000   
         hit_rate       0.986631    0.936594     0.960145    0.521739   
         ppv            0.978780    0.727145     0.596847    0.842105   
         f1             0.982690    0.818686     0.736111    0.644295   
         RTO           -0.029810    1.292070     1.201887    2.753472   
         RTD            1.221612    1.174271     1.223510    3.872758   

pred                        idvt     engbert          nh    remodnav  
event    metric                                                       
fixation num_gt       563.000000  563.000000  563.000000  563.000000  
         num_pred     347.000000  749.000000  481.000000  263.000000  
         num_matches  221.000000  552.000000  422.000000  173.000000  
         hit_rate       0.392540    0.980462    0.749556    0.307282  
         ppv            0.636888    0.736983    0.877339    0.657795  
         f1             0.485714    0.841463    0.808429    0.418886  
         RTO          -12.058824   -1.443841   -0.251185    1.543353  
         RTD            5.842672    3.735906    5.854753    4.269075  
saccade  num_gt       552.000000  552.000000  552.000000  552.000000  
         num_pred     354.000000  762.000000  453.000000  232.000000  
         num_matches  292.000000  547.000000  424.000000  229.000000  
         hit_rate       0.528986    0.990942    0.768116    0.414855  
         ppv            0.824859    0.717848    0.935982    0.987069  
         f1             0.644592    0.832572    0.843781    0.584184  
         RTO            3.349315   -0.711152   -5.103774   -1.847162  
         RTD            1.722504    1.182105    2.664155    2.171320

In [7]:
ra_offset_alignments = alignment_metrics.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_offset_alignments = ra_offset_alignments.reindex(
    columns=u.sort_labelers(ra_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_offset_alignments

pred                          MN         ivt         ivvt         idt  \
event    metric                                                         
fixation num_gt       391.000000  563.000000   563.000000  563.000000   
         num_pred     404.000000  696.000000  1042.000000  358.000000   
         num_matches  384.000000  528.000000   552.000000  313.000000   
         hit_rate       0.982097    0.937833     0.980462    0.555950   
         ppv            0.950495    0.758621     0.529750    0.874302   
         f1             0.966038    0.838761     0.687850    0.679696   
         RTO           -0.005208    1.267045     0.407609    2.958466   
         RTD            1.421551    1.539472     1.646960    3.661587   
saccade  num_gt       374.000000  552.000000   552.000000  552.000000   
         num_pred     377.000000  711.000000   888.000000  342.000000   
         num_matches  368.000000  514.000000   527.000000  288.000000   
         hit_rate       0.983957    0.931159     0.954710    0.521739   
         ppv            0.976127    0.722925     0.593468    0.842105   
         f1             0.980027    0.813935     0.731944    0.644295   
         RTO           -0.701087    2.632296     1.728653   -6.996528   
         RTD            2.850019    4.989386     4.322001    3.623322   

pred                        idvt     engbert          nh    remodnav  
event    metric                                                       
fixation num_gt       563.000000  563.000000  563.000000  563.000000  
         num_pred     347.000000  749.000000  481.000000  263.000000  
         num_matches  313.000000  557.000000  433.000000  125.000000  
         hit_rate       0.555950    0.989343    0.769094    0.222025  
         ppv            0.902017    0.743658    0.900208    0.475285  
         f1             0.687912    0.849085    0.829502    0.302663  
         RTO            2.958466   -0.775583   -4.866051   -2.576000  
         RTD            3.661587    1.426190    3.366123    4.420433  
saccade  num_gt       552.000000  552.000000  552.000000  552.000000  
         num_pred     354.000000  762.000000  453.000000  232.000000  
         num_matches  292.000000  545.000000  415.000000  227.000000  
         hit_rate       0.528986    0.987319    0.751812    0.411232  
         ppv            0.824859    0.715223    0.916115    0.978448  
         f1             0.644592    0.829528    0.825871    0.579082  
         RTO           -6.883562    5.130275    3.093976    7.004405  
         RTD            3.937892    5.377109    5.060438    5.888462

#### Annotator MN

In [8]:
mn_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('MN', level=u.GT_STR).dropna(axis=1, how='all')
mn_onset_alignments = mn_onset_alignments.reindex(
    columns=u.sort_labelers(mn_onset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

mn_onset_alignments

pred                          RA         ivt        ivvt         idt  \
event    metric                                                        
fixation num_gt       404.000000  404.000000  404.000000  404.000000   
         num_pred     391.000000  502.000000  729.000000  257.000000   
         num_matches  375.000000  380.000000  395.000000  175.000000   
         hit_rate       0.928218    0.940594    0.977723    0.433168   
         ppv            0.959079    0.756972    0.541838    0.680934   
         f1             0.943396    0.838852    0.697264    0.529501   
         RTO            0.917333   -3.776316   -0.126582  -12.902857   
         RTD            4.023411    4.779888    4.277013    6.345009   
saccade  num_gt       377.000000  377.000000  377.000000  377.000000   
         num_pred     374.000000  518.000000  656.000000  246.000000   
         num_matches  369.000000  358.000000  368.000000  197.000000   
         hit_rate       0.978780    0.949602    0.976127    0.522546   
         ppv            0.986631    0.691120    0.560976    0.800813   
         f1             0.982690    0.800000    0.712488    0.632424   
         RTO            0.029810    1.270950    1.184783    2.944162   
         RTD            1.221612    1.301823    1.320238    3.606527   

pred                        idvt     engbert          nh    remodnav  
event    metric                                                       
fixation num_gt       404.000000  404.000000  404.000000  404.000000  
         num_pred     247.000000  539.000000  327.000000  171.000000  
         num_matches  174.000000  400.000000  293.000000  116.000000  
         hit_rate       0.430693    0.990099    0.725248    0.287129  
         ppv            0.704453    0.742115    0.896024    0.678363  
         f1             0.534562    0.848356    0.801642    0.403478  
         RTO          -13.040230   -1.315000   -0.177474    2.077586  
         RTD            6.098199    4.068264    5.795953    4.191845  
saccade  num_gt       377.000000  377.000000  377.000000  377.000000  
         num_pred     255.000000  553.000000  303.000000  150.000000  
         num_matches  200.000000  373.000000  279.000000  150.000000  
         hit_rate       0.530504    0.989390    0.740053    0.397878  
         ppv            0.784314    0.674503    0.920792    1.000000  
         f1             0.632911    0.802151    0.820588    0.569260  
         RTO            3.385000   -0.678284   -5.340502   -2.013333  
         RTD            1.966412    1.235527    2.818686    2.006611

In [9]:
mn_offset_alignments = alignment_metrics.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('MN', level=u.GT_STR).dropna(axis=1, how='all')
mn_offset_alignments = mn_offset_alignments.reindex(
    columns=u.sort_labelers(mn_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

mn_offset_alignments

pred                          RA         ivt        ivvt         idt  \
event    metric                                                        
fixation num_gt       404.000000  404.000000  404.000000  404.000000   
         num_pred     391.000000  502.000000  729.000000  257.000000   
         num_matches  384.000000  385.000000  397.000000  226.000000   
         hit_rate       0.950495    0.952970    0.982673    0.559406   
         ppv            0.982097    0.766932    0.544582    0.879377   
         f1             0.966038    0.849890    0.700794    0.683812   
         RTO            0.005208    1.241558    0.342569    3.013274   
         RTD            1.421551    1.368035    1.639723    3.185254   
saccade  num_gt       377.000000  377.000000  377.000000  377.000000   
         num_pred     374.000000  518.000000  656.000000  246.000000   
         num_matches  368.000000  353.000000  364.000000  198.000000   
         hit_rate       0.976127    0.936340    0.965517    0.525199   
         ppv            0.983957    0.681467    0.554878    0.804878   
         f1             0.980027    0.788827    0.704743    0.635634   
         RTO            0.701087    3.396601    2.123626   -6.186869   
         RTD            2.850019    5.706335    5.254364    4.482337   

pred                        idvt     engbert          nh    remodnav  
event    metric                                                       
fixation num_gt       404.000000  404.000000  404.000000  404.000000  
         num_pred     247.000000  539.000000  327.000000  171.000000  
         num_matches  226.000000  401.000000  307.000000   77.000000  
         hit_rate       0.559406    0.992574    0.759901    0.190594  
         ppv            0.914980    0.743970    0.938838    0.450292  
         f1             0.694316    0.850477    0.839945    0.267826  
         RTO            3.013274   -0.698254   -4.960912   -1.987013  
         RTD            3.185254    1.463171    3.751886    4.075564  
saccade  num_gt       377.000000  377.000000  377.000000  377.000000  
         num_pred     255.000000  553.000000  303.000000  150.000000  
         num_matches  200.000000  371.000000  275.000000  147.000000  
         hit_rate       0.530504    0.984085    0.729443    0.389920  
         ppv            0.784314    0.670886    0.907591    0.980000  
         f1             0.632911    0.797849    0.808824    0.557875  
         RTO           -6.140000    6.064690    3.745455    8.176871  
         RTD            4.521106    5.667434    4.828017    5.824766

## Statistical Analysis
### (1) Fixations

In [10]:
fix_statistics, fix_pvalues, fix_dunns, fix_Ns = ctd.kruskal_wallis_dunns(fix_thresholded_time_diffs, [GT1, GT2], multi_comp=MULTI_COMP)
fix_statistics.index.names = fix_pvalues.index.names = fix_dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pd.concat([fix_statistics, fix_pvalues, fix_pvalues <= ALPHA], axis=1, keys=['H', 'p', 'is_sig']).stack(1, future_stack=True)

H              p  is_sig
channel_type gt                                    
offset       MN  1311.200108  4.074620e-280    True
             RA  1932.148168   0.000000e+00    True
onset        MN   742.128974  4.885965e-157    True
             RA   980.160995  1.745909e-208    True

#### Post Hoc Analysis

In [11]:
fix_post_hoc_onset = ctd.post_hoc_table(fix_dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
fix_post_hoc_onset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --     ***     ***     ***     ***     ***      ***
         RA      --     ***     ***     ***     ***     ***      ***
ivvt     MN  0.0000      --     ***     ***       *    n.s.      ***
         RA  0.0000      --     ***     ***       *    n.s.      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0171  0.0000  0.0000      --    n.s.      ***
         RA  0.0000  0.0224  0.0000  0.0000      --      **      ***
nh       MN  0.0000  1.0000  0.0000  0.0000  0.1378      --      ***
         RA  0.0000  1.0000  0.0000  0.0000  0.0034      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0000  0.0001       --

In [12]:
fix_post_hoc_offset = ctd.post_hoc_table(fix_dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
fix_post_hoc_offset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --     ***     ***     ***     ***     ***      ***
         RA      --     ***     ***     ***     ***     ***      ***
ivvt     MN  0.0000      --     ***     ***     ***     ***      ***
         RA  0.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***     n.s.
         RA  0.0000  0.0000  0.0000  0.0000      --     ***        *
nh       MN  0.0000  0.0000  0.0000  0.0000  0.0000      --       **
         RA  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  1.0000  0.0010       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0267  0.0005       --

### (2) Saccades

In [13]:
sac_statistics, sac_pvalues, sac_dunns, sac_Ns = ctd.kruskal_wallis_dunns(sac_thresholded_time_diffs, [GT1, GT2], multi_comp=MULTI_COMP)
sac_statistics.index.names = sac_pvalues.index.names = sac_dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pd.concat([sac_statistics, sac_pvalues, sac_pvalues <= ALPHA], axis=1, keys=['H', 'p', 'is_sig']).stack(1, future_stack=True)

H              p  is_sig
channel_type gt                                    
offset       MN   831.452931  2.460168e-176    True
             RA  1346.109783  1.128070e-287    True
onset        MN  1411.276030  8.763320e-302    True
             RA  2187.687345   0.000000e+00    True

#### Post Hoc Analysis

In [14]:
sac_post_hoc_onset = ctd.post_hoc_table(sac_dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
sac_post_hoc_onset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***     ***      ***
         RA      --    n.s.     ***     ***     ***     ***      ***
ivvt     MN  1.0000      --     ***     ***     ***     ***      ***
         RA  1.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***        *
         RA  0.0000  0.0000  0.0000  0.0000      --     ***       **
nh       MN  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
         RA  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0233  0.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0050  0.0000       --

In [15]:
sac_post_hoc_offset = ctd.post_hoc_table(sac_dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
sac_post_hoc_offset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***    n.s.      ***
         RA      --    n.s.     ***     ***     ***    n.s.      ***
ivvt     MN  0.4486      --     ***     ***     ***      **      ***
         RA  0.6592      --     ***     ***     ***      **      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --      **        †
         RA  0.0000  0.0000  0.0000  0.0000      --     ***        *
nh       MN  1.0000  0.0091  0.0000  0.0000  0.0045      --      ***
         RA  0.9692  0.0011  0.0000  0.0000  0.0004      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0537  0.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0162  0.0000       --

## Figures
RTO is the Distribution's Mean. RTD is the Distribution's STD.

In [16]:
W, H = 600, 400

### (1) Fixation Temporal (Mis-)Alignment Distribution

In [17]:
fix_temporal_alignment_fig = ctd.distributions_figure(
    fix_thresholded_time_diffs, GT1, gt2=GT2, only_box=False, show_other_gt=True, share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_vspace=0.15,
)
fix_temporal_alignment_fig.update_traces(width=0.95)     # make violins wider so there's less space between them

fix_temporal_alignment_fig.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    yaxis2=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
    showlegend=False,
)

# update subplot titles:
for annotation in fix_temporal_alignment_fig.layout.annotations:
    annotation.text = f"Fixation {annotation.text}"
# temporal_alignment_fig.layout.annotations = []    # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 4, 'A', False
# save_fig(fix_temporal_alignment_fig, FIG_ID, PANEL_ID, 'temporal-alignment_fixation', IS_SUPP)
fix_temporal_alignment_fig.show()

### (2) Saccade Temporal (Mis-)Alignment Distribution

In [18]:
sac_temporal_alignment_fig = ctd.distributions_figure(
    sac_thresholded_time_diffs, GT1, gt2=GT2, only_box=False, show_other_gt=True, share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_vspace=0.15,
)
sac_temporal_alignment_fig.update_traces(width=0.95)     # make violins wider so there's less space between them

sac_temporal_alignment_fig.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    yaxis2=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
    showlegend=False,
)

# update subplot titles:
for annotation in sac_temporal_alignment_fig.layout.annotations:
    annotation.text = f"Saccade {annotation.text}"
# temporal_alignment_fig.layout.annotations = []    # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 4, 'B', False
# save_fig(sac_temporal_alignment_fig, FIG_ID, PANEL_ID, 'temporal-alignment_saccade', IS_SUPP)
sac_temporal_alignment_fig.show()

### Final Figures
##### (1) Split double-sided violins to separate figures (per GT annotator)
##### (2) Merge fixation & saccade figures
##### (3) Convert "standing" violins to ridge-plots

In [19]:
WIDTH, HEIGHT = 1600, 900
SUBPLOT_TITLES = ["Fixation Onset", "Saccade Onset", "Fixation Offset", "Saccade Offset"]

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)

In [20]:
def create_annotator_ridge_plot(fix_fix: go.Figure, sac_fig: go.Figure, gt: str) -> go.Figure:
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=SUBPLOT_TITLES,
        vertical_spacing=0.05, horizontal_spacing=0.02,
        shared_xaxes=True, shared_yaxes=True,
    )
    for ttl in SUBPLOT_TITLES:
        event, boundary = ttl.split(" ")
        old_fig = fix_fix if event.lower().startswith("fix") else sac_fig
        fig = _convert_traces(old_fig, fig, gt, event, boundary)

    # update x axes
    fig.for_each_xaxis(lambda xax: xax.update(
        tickfont=AXIS_TICK_FONT,
        showline=False, showgrid=False, gridcolor='lightgray', gridwidth=1,
        zeroline=True, zerolinecolor='lightgray', zerolinewidth=2,
    ))
    for c in range(2):
        fig.update_xaxes(row=2, col=c+1, title=dict(text="Δt (samples)", font=AXIS_LABEL_FONT, standoff=2),)

    # update y axes
    fig.for_each_yaxis(lambda yax: yax.update(
        showline=False, zeroline=False, showgrid=True, gridcolor='lightgray', gridwidth=1, tickfont=AXIS_TICK_FONT,
    ))
    for r in range(2):
        fig.update_yaxes(row=r+1, col=1, title=dict(text="Detector", font=AXIS_LABEL_FONT, standoff=4),)

    # update subtitles
    fig.for_each_annotation(lambda ann: ann.update(font=TITLE_FONT, textangle=0,))

    fig.update_layout(
        font_family=FONT_FAMILY,
        width=WIDTH, height=HEIGHT,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=25, b=0, pad=0),
        showlegend=False,
    )
    return fig

def _convert_traces(old_fig: go.Figure, new_fig: go.Figure, gt: str, event: str, boundary: str) -> go.Figure:
    event, boundary = event.lower(), boundary.lower()
    row = 1 if "onset" in boundary else 2
    col = 1 if "fixation" in event else 2
    other = "MN" if gt == "RA" else "RA"
    for trace in old_fig.data:
        if not trace["name"].startswith(gt):
            # ignore data from other annotator
            continue
        if trace["scalegroup"] != boundary:
            # ignore data from other boundary type
            continue
        new_tr = copy.deepcopy(trace)

        # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
        if new_tr["x0"] == "Other GT":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
        elif new_tr["x0"].startswith("i"):
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
        elif new_tr["x0"] == "remodnav":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
        new_tr['x'] = new_tr['y']

        # reset irrelevant props
        new_tr['y'] = new_tr['x0'] = None
        new_tr["showlegend"] = False
        new_tr["side"] = "positive"
        new_tr["width"] = 1.8
        new_tr["box"] = dict(visible=False, width=0.95, line=dict(width=1))
        new_tr["meanline"] = dict(visible=True, width=3, color='lightgray')
        new_tr["opacity"] = 1
        new_tr["visible"] = True

        # add the trace to the correct subplot
        new_fig.add_trace(new_tr, row=row, col=col)
    return new_fig

In [21]:
NAME = "fig3_ra"
ra_fig = create_annotator_ridge_plot(fix_temporal_alignment_fig, sac_temporal_alignment_fig, "RA")

ra_fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# ra_fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
ra_fig.show()

In [22]:
NAME = "fig3_mn"
mn_fig = create_annotator_ridge_plot(fix_temporal_alignment_fig, sac_temporal_alignment_fig, "MN")

mn_fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# mn_fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
mn_fig.show()

## Repeat Analysis
### Repeating the analysis with subset of image-stimulus trials that were recorded @ 500Hz

### (1) Fixations

In [23]:
fix_time_diffs = ctd.load(DATASET_NAME, PROCESSED_DATA_DIR, label=1, stimulus_type=STIMULUS_TYPE,)
fix_subset = fix_time_diffs.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)
fix_subset_thresholded = fix_subset.map(lambda vals: np.fromiter(filter(lambda v: abs(v) <= MATCHING_THRESHOLD, vals), dtype=int))

fix_sub_statistics, fix_sub_pvalues, fix_sub_dunns, fix_sub_Ns = ctd.kruskal_wallis_dunns(fix_subset_thresholded, [GT1, GT2], multi_comp=MULTI_COMP)
fix_sub_statistics.index.names = fix_sub_pvalues.index.names = fix_sub_dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pd.concat([fix_sub_statistics, fix_sub_pvalues, fix_sub_pvalues <= ALPHA], axis=1, keys=['H', 'p', 'is_sig']).stack(1, future_stack=True)

H              p  is_sig
channel_type gt                                    
offset       MN  1135.036906  5.474292e-242    True
             RA  1523.684817   0.000000e+00    True
onset        MN   673.373198  3.426648e-142    True
             RA   813.500047  1.864103e-172    True

In [24]:
fix_sub_post_hoc_onset = ctd.post_hoc_table(fix_sub_dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
fix_sub_post_hoc_offset = ctd.post_hoc_table(fix_sub_dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)

print("Fixation Onset Post Hoc")
display(fix_sub_post_hoc_onset)

print("Fixation Offset Post Hoc")
display(fix_sub_post_hoc_offset)

Fixation Onset Post Hoc


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --     ***     ***     ***     ***     ***      ***
         RA      --     ***     ***     ***     ***     ***      ***
ivvt     MN  0.0000      --     ***     ***     ***    n.s.      ***
         RA  0.0000      --     ***     ***      **    n.s.      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0007  0.0000  0.0000      --    n.s.      ***
         RA  0.0000  0.0034  0.0000  0.0000      --      **      ***
nh       MN  0.0000  1.0000  0.0000  0.0000  0.0933      --      ***
         RA  0.0000  1.0000  0.0000  0.0000  0.0019      --       **
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000       --
         RA  0.0000  0.0001  0.0000  0.0000  0.0000  0.0012       --

Fixation Offset Post Hoc


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --     ***     ***     ***     ***     ***      ***
         RA      --     ***     ***     ***     ***     ***      ***
ivvt     MN  0.0000      --     ***     ***     ***     ***      ***
         RA  0.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***     n.s.
         RA  0.0000  0.0000  0.0000  0.0000      --     ***        *
nh       MN  0.0000  0.0000  0.0000  0.0000  0.0000      --       **
         RA  0.0000  0.0000  0.0000  0.0000  0.0000      --       **
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.9587  0.0049       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0228  0.0047       --

### (2) Saccades

In [25]:
sac_time_diffs = ctd.load(DATASET_NAME, PROCESSED_DATA_DIR, label=2, stimulus_type=STIMULUS_TYPE,)
sac_subset = sac_time_diffs.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)
sac_subset_thresholded = sac_subset.map(lambda vals: np.fromiter(filter(lambda v: abs(v) <= MATCHING_THRESHOLD, vals), dtype=int))

sac_sub_statistics, sac_sub_pvalues, sac_sub_dunns, sac_sub_Ns = ctd.kruskal_wallis_dunns(sac_subset_thresholded, [GT1, GT2], multi_comp=MULTI_COMP)
sac_sub_statistics.index.names = sac_sub_pvalues.index.names = sac_sub_dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pd.concat([sac_sub_statistics, sac_sub_pvalues, sac_sub_pvalues <= ALPHA], axis=1, keys=['H', 'p', 'is_sig']).stack(1, future_stack=True)

H              p  is_sig
channel_type gt                                    
offset       MN   771.871672  1.838429e-163    True
             RA  1179.753239  1.152933e-251    True
onset        MN  1215.150071  2.518339e-259    True
             RA  1738.223271   0.000000e+00    True

In [26]:
sac_sub_post_hoc_onset = ctd.post_hoc_table(sac_sub_dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
sac_sub_post_hoc_offset = ctd.post_hoc_table(sac_sub_dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)

print("Saccade Onset Post Hoc")
display(sac_sub_post_hoc_onset)

print("Saccade Offset Post Hoc")
display(sac_sub_post_hoc_offset)

Saccade Onset Post Hoc


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***     ***      ***
         RA      --    n.s.     ***     ***     ***     ***      ***
ivvt     MN  1.0000      --     ***     ***     ***     ***      ***
         RA  1.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***       **
         RA  0.0000  0.0000  0.0000  0.0000      --     ***       **
nh       MN  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
         RA  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0082  0.0002       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0014  0.0000       --

Saccade Offset Post Hoc


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***    n.s.      ***
         RA      --    n.s.     ***     ***     ***    n.s.      ***
ivvt     MN  0.5770      --     ***     ***     ***       *      ***
         RA  0.8434      --     ***     ***     ***      **      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --      **        *
         RA  0.0000  0.0000  0.0000  0.0000      --      **       **
nh       MN  1.0000  0.0447  0.0000  0.0000  0.0085      --      ***
         RA  1.0000  0.0039  0.0000  0.0000  0.0026      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0118  0.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0064  0.0000       --